In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !git clone https://github.com/cs357/demos-cs357.git
    !mv demos-cs357/figures/ .
    !mv demos-cs357/additional_files/ .

# Solving linear system of equations

Useful libraries for this notebook:

In [ ]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla
import scipy.sparse as sparse

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

import seaborn as sns
sns.set(font_scale=2)
plt.style.use('seaborn-whitegrid')

from time import time


## 1) Transforming images using linear operators:

### Import an image of a Social Security Number

In [ ]:
img = Image.open('figures/ssn.png')
xmat = (255 - np.asarray(img).max(axis=2))/255

In [ ]:
print(xmat.shape)
print(xmat.min(),xmat.max())

In [ ]:
plt.imshow(xmat);

### "Vectorize" the image, creating the 1d array `x`

In [ ]:
x = xmat.flatten()
x.shape

### Construct a "blur" matrix
More about this blur matrix on a later MP...

In [ ]:
imat, jmat = np.meshgrid(np.arange(xmat.shape[0]), np.arange(xmat.shape[1]), indexing='ij')

ivec = np.atleast_2d(imat.flatten())
jvec = np.atleast_2d(jmat.flatten())

A = np.fmax(0, 1 - np.sqrt((ivec.T - ivec)**2 + (jvec.T - jvec)**2)/5)
A /= A.sum(axis=1)

### Compute b = A x

In [ ]:
b = A @ x

In [ ]:
b2D=b.reshape(xmat.shape)
plt.imshow(b2D)

### Assume we have the blurred image (b), solve for the unblurred one (x)

In essence, we want $x = A^{-1} b$. Instead of evaluating the inverse, we can evaluate:

#### a)  `np.linalg.solve`:

In [ ]:
x_solve1 = la.solve(A,b)

In [ ]:
plt.imshow(x_solve1.reshape(xmat.shape))

Voila! We have our recovered original image :-)

#### What is happening inside "solve"?

A factorization and substitutions. For example, let's think of LU factorization:

If $Ax = P L U x = b$, then there are two steps:
1. $y \leftarrow \text{solve}\,\, L y = P^Tb$
2. $x \leftarrow \text{solve}\,\, U x = y$

#### b) LU factorization and triangular substitutions:

In [ ]:
import scipy.linalg as sla
P, L, U = sla.lu(A)

In [ ]:
# L y = P^T b
y = sla.solve_triangular(L, P.T@b, lower=True)

# U x = y
x_solve = sla.solve_triangular(U, y)

plt.imshow(x_solve.reshape(xmat.shape))

#### Why not just `np.linalg.solve`?

Suppose you have many social security numbers to un-blur. You should factorize your blur matrix only once and then perform several triangular solves.

Let's time:

1. solve including factorization
2. factorization
3. solve, given a factorization

In [ ]:
%timeit sla.solve(A, b)

In [ ]:
%timeit P, L, U = sla.lu(A)

In [ ]:
%timeit sla.solve_triangular(U, y)

### Let's take a look at the matrix format

In [ ]:
plt.figure()
plt.subplot(131)
plt.spy(A); plt.xticks([]); plt.yticks([]);

plt.subplot(132)
plt.spy(L); plt.xticks([]); plt.yticks([]);

plt.subplot(133)
plt.spy(U); plt.xticks([]); plt.yticks([]);

Note that many entries of the matrices are zeros! We don't need to store all of the zeros. In fact, we can even use solvers that efficiently take into account the format of the matrices.

In [ ]:
 A_csr = sparse.csr_matrix(A)

In [ ]:
%timeit sparse.linalg.spsolve(A_csr,b)

In [ ]:
x_solve_3 = sparse.linalg.spsolve(A_csr,b)
plt.imshow(x_solve_3.reshape(xmat.shape))

This is just a teaser :-) We will talk more later about sparse systems.

### Now add some noise

In [ ]:
b_noisy = b + 1e-5 * np.random.rand(b.size)

y = sla.solve_triangular(L, np.dot(P.T, b_noisy), lower=True)
x_solve = sla.solve_triangular(U, y)

plt.imshow(x_solve.reshape(xmat.shape))


What happened to your solution. We will soon be talking about conditioning of matrices. 

##  2) Relative cost of matrix operations

In [ ]:
n_values = (10**np.linspace(1, 4, 15)).astype(np.int32)
n_values

In [ ]:
times_matmul = []
times_lu = []

for n in n_values:
    print(n)
    A = np.random.randn(n, n)
    start_time = time()
    A.dot(A)
    times_matmul.append(time() - start_time)
    start_time = time()
    sla.lu(A)
    times_lu.append(time() - start_time)

In [ ]:
plt.plot(n_values, times_matmul, label='matmul')
plt.plot(n_values, times_lu, label='lu')
plt.legend(loc="best")
plt.xlabel("Matrix size $n$")
plt.ylabel("Wall time [s]")

* The faster algorithms make the slower ones look bad. But... it's all relative.
* Can we get a better plot?
* Can we see the asymptotic cost ($O(n^3)$) of these algorithms from the plot?

In [ ]:
plt.loglog(n_values, times_matmul, label='matmul')
plt.loglog(n_values, times_lu, label='lu')
plt.legend(loc="best")
plt.xlabel("Matrix size $n$")
plt.ylabel("Wall time [s]")

## 3) Example:  Topology design optimization

![](figures\myfigure.png)

When performing optimization of structural problem, for example to obtain the bridge design above, you may want to use a numerical method called Finite Element Method. The optimization will consist of a series of `solve` of the form:
$$ {\bf K} {\bf u} = {\bf F} $$

Here will load the matrix $ {\bf K}$ from a file. The matrix is given in Compressed Sparse Column (CSC) format.

In [ ]:
K = sparse.load_npz('additional_files/yourmatrix.npz')
K

We can `spy` the distribution of the non-zero entries of the matrix:

In [ ]:
plt.spy(K)
plt.show()

The matrix ${\bf K}$ has a banded format, and it is also symmetric and positive definite. 

In [ ]:
Kdense = K.todense()

In [ ]:
Kdense.shape

In [ ]:
np.max(Kdense-Kdense.T)

In [ ]:
sla.norm(Kdense-Kdense.T)

### Solving the linear system of equations using different methods:

In [ ]:
F = np.zeros(K.shape[0])
F[1]=-1

#### a) la.solve

In [ ]:
u1 = sla.solve(Kdense,F)
u1.shape

In [ ]:
%timeit sla.solve(Kdense,F)

#### b) LU factorization

In [ ]:
P,L,U = sla.lu(Kdense)
y = sla.solve_triangular(L, np.dot(P.T, F), lower=True)
u2 = sla.solve_triangular(U, y)
u2.shape

In [ ]:
%timeit sla.lu(Kdense)
%timeit sla.solve_triangular(L, np.dot(P.T, F), lower=True)
%timeit sla.solve_triangular(U, y)

#### c) Cholesky factorization

In [ ]:
Kcho = sla.cholesky(Kdense)
u3 = sla.cho_solve((Kcho,False),F)
u3.shape

In [ ]:
%timeit sla.cholesky(Kdense)
%timeit sla.cho_solve((Kcho,False),F)

#### d) Sparse solve

In [ ]:
from scipy.sparse.linalg import spsolve

In [ ]:
u4 = spsolve(K,F)
u4.shape

In [ ]:
%timeit spsolve(K,F)

## 4) Steady-State Advection Diffusion Equation

In [ ]:
import sys
sys.path.append('./additional_files')
from CA_4_support import *
from matplotlib import animation,rc
from IPython.display import HTML


Consider the following ordinary differential equation (ODE) for $x$ in the interval $[-1,1]$:

$$-u''(x) + u'(x) = f(x)$$

With boundary conditions

$$u(-1) = u(1) = 0$$
           
Here, $f(x)$ is a known function and $u(x)$ is the unknown that we are trying to solve for.

### Spectral Element Approximation
The spectral element method (SEM) is a way to solve such an ODE numerically.  In the simplest case, SEM approximates $u(x)$ by a polynomial of degree $N$.

A polynomial can be uniquely described by $P = N+1$ points on the interval $[-1,1]$.  So the SEM introduces a set of $P$ discrete points

$-1 = x_0 < x_1 < x_2 < \dots < x_{N-1} < x_N = 1$

and constructs a linear system

$\mathbf{A}\mathbf{u} = \mathbf{b}$

that is solved to obtain the value of $u$ at each point $x_i$. Here, the vector $\mathbf{u} = [u_1,u_2,\dots,u_{N-1}]^T$, where $u_i = u(x_i)$ for $i = 1,2,\dots,N-1$.

Note that we exclude the points $x_0=-1$ and $x_N = 1$ because we already know the value of $u_0$ and $u_N$ from the boundary conditions.

$\mathbf{b}$ is a vector that depends on the function $f$.

We'll provide a function that takes a polynomial order $N$ as input and creates both the partition points $x_i$ defined in the interval $([-1,1]$, and the system matrix $\mathbf{A}$.  We'll also provide a function that takes the  points $x_i$ and a known function $f(x)$ and returns the vector $\mathbf{b}$. The function signatures are defined below:

```python
A,x =  SEM_system_1(N)

def f(z):
    # define the function f as a function of z
    return f
  
b = SEM_rhs_1(f,x)
```
Generate the arrays $\mathbf{A}$, $\mathbf{b}$, and $\mathbf{x}$ using the provided functions. Use for example `N = 30`

In [ ]:
N = 30  # polynomial order
A,x = SEM_system_1(N)

def f(z):
    return 2*(z-1)

b = SEM_rhs_1(f,x)

Check out the shapes of all 3 objects

The array of points $\mathbf{x}$ is of size $N+1$ but the matrix and right hand side lead to a linear system of size $(N-1) \times (N-1)$.  This is because we don't need to solve for the value of $u$ at the two endpoints.

Use $\mathbf{A}$, $\mathbf{b}$, and $\mathbf{x}$ to solve for $\mathbf{u}$.  Plot your numerical approximation and the exact solution $u_e(x) = x^2 - 1$.  Don't forget to add zeros to the start and end of the vector to account for the boundary conditions.  Do the two solutions match up?

In [ ]:
#clear
u = ...
u_plot = ...
u_exact = x**2 - 1

In [ ]:
#clear
plt.plot(x,u_plot,'o')
plt.plot(x,u_exact,'r')

You can run the cells above changing the number of points. Use larger and smaller values. What do you notice?

The solution will change for different right hand sides.  Experiment with different functions $f(x)$ and see what the solution looks like. For example, try the function:

$$ f(x) = \sin(1.5 \pi x) $$

Note that since your ODE has not changed, your system matrix  $\mathbf{A}$ is still the same! You only need to redefine the vector $\mathbf{b}$.

In [ ]:
def f(z):
    return ...

b = SEM_rhs_1(f,x)

In [ ]:
#Obtain u and plot 
u =...
u_plot = ...

plt.plot(x,u_plot,'-o')

## 5) Time-Dependent Advection Diffusion

Now we'll look at the time-dependent advection diffusion equation, which is a partial differential equation (PDE)

$$\frac{\partial u}{\partial t} + \frac{\partial u}{\partial x} - D\frac{\partial^2 u}{\partial x^2} = 0$$

with boundary conditions:

$$u(-1,t) = 0 = \frac{\partial u}{\partial x}(1,t)$$

Where $D = 0.1$ is the diffusion coefficient.  Notice how the boundary condition at the right hand side is a little different than the first equation.

In order to solve this equation we again use the SEM.  However, the equation is time dependent, so we also partition the time dimension:

$$0 = t_0 < t_1 < t_2 < \dots$$.

We assume the $t_i$'s are evenly spaced with distance $\Delta t$.

Now, the solution $u$ depends on both space and time: $u(x,t)$.  For each time $t_n$, we can define a vector of values of $u$ like above:

$\mathbf{u}^{(n)} = [u_1^{(n)},\dots, u_N^{(n)}]^T$.  Here $u_i^{(n)} = u(x_i,t_n)$.  We assume we know $u(x,0)$ so that $\mathbf{u}^{(0)}$ is known.  Then we get a *timestepping scheme*:

\begin{align}
\mathbf{A}\mathbf{u}^{(1)} &= \mathbf{b}^{(0)}\\
\mathbf{A}\mathbf{u}^{(2)} &= \mathbf{b}^{(1)}\\
\mathbf{A}\mathbf{u}^{(3)} &= \mathbf{b}^{(2)}\\
&\ \vdots\\
\mathbf{A}\mathbf{u}^{(n+1)} &= \mathbf{b}^{(n)}
\end{align}

Here $\mathbf{b}^{(n)}$ is a vector that depends on $\mathbf{u}^{(n)}$, and can be obtained using the helper function:
```
b = SEM_rhs_2(un)
```
We'll select a value of $\Delta t = 0.01$ and take 200 timesteps.

In [ ]:
dt = 0.01        # time increment
timesteps = 200  # number of time steps
N = 50           # polynomial order

The matrix $\mathbf{A}$ depends not just on the polynomial order $N$ but also the value of $\Delta t$. We provide the helper function:

```python
A,x = SEM_system_2(N,dt)
```
Generate the arrays $\mathbf{A}$, and $\mathbf{x}$.

In [ ]:
A,x = SEM_system_2(N,dt)

We also provide a function that defines the initial condition $u(x,0)$.  

In [ ]:
def u_initial(z):
    a = -10*(z+1)*z**6*(z-1)*(z< 0)
    return a

We will store all the vectors $\mathbf{u}^{(0)}$, $\mathbf{u}^{(1)}$, $\mathbf{u}^{(2)}$,$\dots$ as columns of a matrix. Initialize with zeros the 2d numpy array (matrix) $\mathbf{u}$ with the appropriate shape. Update the first column with the initial condition using the function `u_initial`.

In [ ]:
u = np.zeros((x.shape[0],timesteps+1))
# store the initial condition in the zeroth column
u[:,0] = u_initial(x)

In [ ]:
# plot initial condition
plt.plot(x,u[:,0],'o-')
plt.xlabel('$x$')
plt.ylabel('$u$')
plt.title('Initial Condition')
plt.show()

Generate the array $\mathbf{b}$ corresponding to the initial condition `u[:,0]`

In [ ]:
# First time step
u_old = u[:,0]
b = SEM_rhs_2(u_old)

Let's take the first time step and store the solution in `u[:,1]`. Make sure you are handling the size of your arrays correctly and including the boundary condition

In [ ]:
u_new = np.linalg.solve(A,b)
u[:,1] = np.append(0,u_new)   # add back boundary condition

Now you can take the second time step, and store your solution in `u[:,2]`

In [ ]:
# second time step
u_old = u[:,1]
b = SEM_rhs_2(u_old)
u_new = np.linalg.solve(A,b)
u[:,2] = np.append(0,u_new) # add back boundary condition

Let's plot what you have so far:

In [ ]:
plt.plot(x,u[:,0],label = '$\mathbf{u}^{(0)}$')
plt.plot(x,u[:,1],label ='$\mathbf{u}^{(1)}$' )
plt.plot(x,u[:,2],label = '$\mathbf{u}^{(2)}$')
plt.xlabel('$x$')
plt.ylabel('$u$')
plt.legend()
plt.show()

Only 198 time steps to go...
We'll obviously want to do this in a loop.  But calling `np.linalg.solve` every time step is too expensive.  We'll want to use the LU decomposition of $A$ to make computing the solution less expensive.  Here $\mathbf{A} = \mathbf{PLU}$, where $\mathbf{P}^{-1} = \mathbf{P}^T$ and $\mathbf{L}$ and $\mathbf{U}$ are lower and upper diagonal matrices

The equation

$\mathbf{A}\mathbf{u}^{(n+1)} = \mathbf{b}^{(n)}$

is transformed into 

$\mathbf{PLUu}^{(n+1)} = \mathbf{b}^{(n)}$

so 

$\mathbf{u}^{(n+1)} = \mathbf{U}^{-1}\mathbf{L}^{-1}\mathbf{P}^T\mathbf{b}^{(n)}$

Compute the LU factorization of $\mathbf{A}$ using `scipy.linalg.lu` and invert $\mathbf{L}$ and $\mathbf{U}$ using `scipy.linalg.solve_triangular`

Check out the documentation for `scipy.linalg.solve_triangular` before you use them.  In particular, pay attention to the argument `lower`, and `unit_diagonal`

Use `plt.spy` to plot the non-zero pattern of each matrix

In [ ]:
plt.figure()
plt.spy(P)
plt.figure()
plt.spy(L)
plt.figure()
plt.spy(U)

We can plot the time history of the solution using a movie:

In [ ]:
fig,ax = plt.subplots()
ax.set_xlim((-1,1))
ax.set_ylim((0,1.0))
line, = ax.plot([],[],lw=2)

def init():
    line.set_data([],[])
    return (line,)

def animate(i):
    y = u[:,i]
    line.set_data(x,y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=200, interval=50, blit=True)
rc('animation',html='html5')
anim

## 6) Time-Dependent Diffusion Equation
Now we'll look at the closely related time-dependent diffusion equation

$$\frac{\partial u}{\partial t} - D\frac{\partial^2 u}{\partial x^2} = 0$$

with boundary conditions:

$$u(-1,t) = 0 = u(1,t)$$

We'll get another time-stepping scheme of the same form:
\begin{align}
\mathbf{A}\mathbf{u}^{(1)} &= \mathbf{b}^{(0)}\\
\mathbf{A}\mathbf{u}^{(2)} &= \mathbf{b}^{(1)}\\
\mathbf{A}\mathbf{u}^{(3)} &= \mathbf{b}^{(2)}\\
&\ \vdots\\
\mathbf{A}\mathbf{u}^{(n+1)} &= \mathbf{b}^{(n)}
\end{align}

We will again define the number of time steps, the time increment, the polynomial degree and use a helper function to construct the matrix $\mathbf{A}$:

In [ ]:
dt = 0.01
timesteps = 200
N = 50
A,x = SEM_system_3(N,dt)

The diffusion equation leads to a *symmetric positive-definite* matrix.  This means that $\mathbf{A} = \mathbf{A}^T$ and the eigenvalues of $\mathbf{A}$ are positive. (You'll learn about eigenvalues next week).

- Check if the matrix is symmetric:
- Check if the matrix has all positive eigenvalues (you can use np.linalg.eigvals(A))

In [ ]:
# check for symmetry
print(np.allclose(A,A.T))

# check that smallest eigenvalue is positive
eig_vals = np.linalg.eigvals(A)
print(np.min(eig_vals) > 0)

Instead of an LU factorization, we'll use the Cholesky factorization: $\mathbf{A} = \mathbf{U}^T\mathbf{U}$, where $\mathbf{U}$ is an upper triangular matrix. This is a more efficient factorization method for symmetric positive-definite matrices) - about half the number of floating operations of LU factorization.

Let's compare the computational time for both methods:

In [ ]:
N = 200
A,x = SEM_system_3(N,dt)

In [ ]:
%timeit sla.cholesky(A)

In [ ]:
%timeit sla.lu(A)

For the time evolution we'll set `N = 50`, we have the following system matrix $\mathbf{A}$ and initial condition:

In [ ]:
N = 50
A,x = SEM_system_3(N,dt)

def u_initial(z):
    a = -10*(z+1)*z**6*(z-1)*(z< 0)
    return a

We will store all the vectors $\mathbf{u}^{(0)}$, $\mathbf{u}^{(1)}$, $\mathbf{u}^{(2)}$,$\dots$ as columns of a matrix. Initialize with zeros the 2d numpy array (matrix) $\mathbf{u}$ with the appropriate shape. Update the first column with the initial condition using the function `u_initial`.

In [ ]:
u = np.zeros((x.shape[0],timesteps+1))

u[:,0] = u_initial(x)

Use `sla.cholesky` and `sla.solve_triangular` to evolve the solution forward

In [ ]:
U = sla.cholesky(A)

for i in range(timesteps):
    u_old = u[:,i]
    b = SEM_rhs_3(u_old)
    b1 = sla.solve_triangular(U.T,b,lower=True,unit_diagonal=False)
    u_new = sla.solve_triangular(U,b1,lower=False,unit_diagonal=False)
    u[:,i+1] = np.append(0,np.append(u_new,0))

Another movie..

In [ ]:
fig,ax = plt.subplots()
ax.set_xlim((-1,1))
ax.set_ylim((0,1.0))
line, = ax.plot([],[],lw=2)

def init():
    line.set_data([],[])
    return (line,)

def animate(i):
    y = u[:,i]
    line.set_data(x,y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

rc('animation',html='html5')
anim